In [13]:
from numpy import array
from keras.preprocessing.text import one_hot
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers.core import Activation, Dropout, Dense
from keras.layers import Flatten, LSTM
from keras.layers import GlobalMaxPooling1D
from keras.models import Model
from keras.layers.embeddings import Embedding
from sklearn.model_selection import train_test_split
from keras.preprocessing.text import Tokenizer
from keras.layers import Input
from keras.layers.merge import Concatenate
from keras.layers import Bidirectional
from keras import optimizers
from keras.layers import TimeDistributed
from time import time
from keras.callbacks import TensorBoard

import pandas as pd
import numpy as np
import re

import matplotlib.pyplot as plt

In [14]:
from keras.models import load_model
model = load_model('avsr_english_v2.h5')

In [15]:
words_list = ['About','Bad','Bottle', 'Come','Cow', 'Good','Pencil', 'Read', 'Where']
import os
import numpy as np
import pandas as pd
import imutils
from imutils import face_utils
import dlib
import cv2
from scipy.fftpack import fft
from scipy import signal
from scipy.io import wavfile
import librosa

SHAPE_PREDICTOR_LOCATION = "./shape_predictor_68_face_landmarks.dat"
face_detector = dlib.get_frontal_face_detector()
shape_predictor = dlib.shape_predictor(SHAPE_PREDICTOR_LOCATION)

X  = []
Y  = []
label_dictionary = dict()


def process_Video(video):
    print("================================")
    X = []
    Y = []
    cap = cv2.VideoCapture(video)
    fps = cap.get(cv2.CAP_PROP_FPS)
    print("FPS : " + str(fps))
    if(not cap.isOpened()):
        print(" .....Cannot open video stream or file!.....")
    print(" video/camera opened...")
    print("********************************************")
    while(cap.isOpened()):
        frameId = cap.get(1)
        try:
            ret, image = cap.read()
            if(ret==False):
                break
            else:
                print("Frame ID :"+ str(frameId))
                #image = imutils.rotate_bound(image,angle=90)
                #image = imutils.resize(image,width = 500)
                gray_image = cv2.cvtColor(image,cv2.COLOR_BGR2GRAY)
                #cv2.imshow("gray",image)
                faces = face_detector(gray_image,1)
                for (i,rect) in enumerate(faces):
                    shape = shape_predictor(gray_image,rect)
                    shape = face_utils.shape_to_np(shape)

                    for (name, (i,j)) in face_utils.FACIAL_LANDMARKS_IDXS.items():
                        if(name=='mouth'):
                            #cpyImage = image.copy()
                            X_frame = []
                            Y_frame = []
                            for (x,y) in   shape[i:j]:
                                X_frame.append(x)
                                Y_frame.append(y)
                    X.append(X_frame)
                    Y.append(Y_frame)

        except Exception as e:
            print("ERROR OCCURED......." + e)
            break;
    cap.release()
    cv2.destroyAllWindows()
    return X,Y

def process_Arrays(X,Y):
    X_sd_array = []
    Y_sd_array = []
    X_sd_array = np.std(X,axis = 0).astype('float32')
    Y_sd_array = np.std(Y,axis = 0).astype('float32')
    return X_sd_array,Y_sd_array


def process(training_Dataframe,video):

    X,Y = process_Video(video)
    X,Y = process_Arrays(X,Y)
    feature_vector = np.r_[X,Y]
    return feature_vector




In [23]:
filetotest = "/media/vinay/Academy/Final_year_project/Rohith_English_mp4_1sec/1_sec_cbn_eng_bottle_4.mp4"
training_Dataframe = pd.DataFrame(dtype='float32')
training_Dataframe = process(training_Dataframe,filetotest)
print(training_Dataframe)

FPS : 30.0
 video/camera opened...
********************************************
Frame ID :0.0
Frame ID :1.0
Frame ID :2.0
Frame ID :3.0
Frame ID :4.0
Frame ID :5.0
Frame ID :6.0
Frame ID :7.0
Frame ID :8.0
Frame ID :9.0
Frame ID :10.0
Frame ID :11.0
Frame ID :12.0
Frame ID :13.0
Frame ID :14.0
Frame ID :15.0
Frame ID :16.0
Frame ID :17.0
Frame ID :18.0
Frame ID :19.0
Frame ID :20.0
Frame ID :21.0
Frame ID :22.0
Frame ID :23.0
Frame ID :24.0
Frame ID :25.0
Frame ID :26.0
Frame ID :27.0
Frame ID :28.0
Frame ID :29.0
[1.3844373  1.1967548  1.6141733  1.5162088  1.7065234  1.7435596
 1.8427033  1.6653328  1.6        1.4255604  1.454495   1.3
 1.569147   1.476106   1.4        1.5807874  2.1931713  1.7113997
 1.605892   1.5648926  1.0241528  0.5587685  0.8406347  0.8465617
 0.8749603  0.5859465  1.0857666  3.0733624  3.9966652  4.2244
 4.012342   3.14625    1.1527745  0.5617433  0.48989794 0.5120764
 1.1100551  2.2472205  2.351359   2.3676054 ]


In [24]:
X_Predict = np.array(training_Dataframe)
X_Predict = X_Predict.reshape(1*8,5)
dat_fi1=np.array([])
for a in range(8):
    for b in range(5):
        if b==0:
            first1 =  X_Predict[a][b]
        else:
            second1 = X_Predict[a][b]
            ec1 = first1 - second1
            dat_fi1 = np.append(dat_fi1,ec1,axis=None)
            first1 =second1

In [25]:
X_Predict = dat_fi1.reshape(1,8,4)


In [26]:
AUDIO_FILE_NAME='teste1.wav'

from moviepy import editor

def convert_vedio_to_audio(video):
    try:
        os.system('ffmpeg -i {} -acodec pcm_s16le -ar 16000 {}'.format(filetotest, AUDIO_FILE_NAME))
        return True,AUDIO_FILE_NAME
    except Exception as e:
       print(e)
       print("conversion from video to audio failed ")
    return False,AUDIO_FILE_NAME
    

def extract_feature(audio_file_name):
    X, sample_rate = librosa.load(audio_file_name,sr = 16000)
    stft = np.abs(librosa.stft(X))
    mfccs = np.mean(librosa.feature.mfcc(y=X, sr=sample_rate, n_mfcc=40).T,axis=0)
    chroma = np.mean(librosa.feature.chroma_stft(S=stft, sr=sample_rate).T,axis=0)
    mel = np.mean(librosa.feature.melspectrogram(X, sr=sample_rate).T,axis=0)
    contrast = np.mean(librosa.feature.spectral_contrast(S=stft, sr=sample_rate).T,axis=0)
    tonnetz = np.mean(librosa.feature.tonnetz(y=librosa.effects.harmonic(X),
    sr=sample_rate).T,axis=0)
        
    return mfccs,chroma,mel,contrast,tonnetz
 

def process_audio_features(mfccs,chroma,mel,contrast,tonnetz):   

        features_array = np.hstack([mfccs,chroma,mel,contrast,tonnetz])
        return features_array
    

def get_audio_feature_vector(video_location):
    isConverted,audio_file_name = convert_vedio_to_audio(video_location)
    if(isConverted):
         mfccs,chroma,mel,contrast,tonnetz =  extract_feature(audio_file_name)
         feature_vector = process_audio_features(mfccs,chroma,mel,contrast,tonnetz)
         feature_vector  = np.around(feature_vector,decimals=3)
         return feature_vector
    else:
       print('Video conversion failed...')

In [27]:
X_Predict_Audio = get_audio_feature_vector(filetotest)
os.remove("teste1.wav")

In [28]:
X_Predict_Audio.reshape(1,193,)

array([[-4.44128e+02,  1.48095e+02,  1.44480e+01,  1.60490e+01,
         6.33100e+00, -1.37000e+00,  3.64700e+00, -3.77400e+00,
        -2.31800e+00,  1.12210e+01, -1.73100e+00,  9.96000e-01,
        -5.88800e+00, -1.94800e+00, -4.83800e+00, -3.61300e+00,
        -9.86200e+00, -1.41500e+00, -7.33400e+00,  4.06000e-01,
        -5.82700e+00, -2.28600e+00, -4.81000e+00, -2.79000e-01,
        -6.78500e+00, -5.02600e+00, -1.84400e+00, -5.28800e+00,
        -5.23000e+00, -1.30000e+00, -3.52200e+00, -5.39000e+00,
        -7.43400e+00, -2.50200e+00, -1.77800e+00, -1.27900e+00,
        -2.05300e+00, -2.54000e+00, -5.38300e+00, -2.17000e+00,
         5.98000e-01,  6.84000e-01,  7.82000e-01,  6.85000e-01,
         5.48000e-01,  6.75000e-01,  5.66000e-01,  4.32000e-01,
         5.31000e-01,  5.97000e-01,  6.51000e-01,  7.25000e-01,
         1.30000e-02,  5.00000e-03,  1.00000e-02,  2.02000e-01,
         4.64700e+00,  1.16700e+00,  1.36000e-01,  2.30000e-02,
         2.80000e-02,  1.36000e-01,  1.8

In [29]:
predict_prob=model.predict([X_Predict_Audio.reshape(1,193,),X_Predict.reshape(1,8,4)])
index=np.argmax(predict_prob[0])
#print(index)
print(words_list[index])

Bottle
